In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics
graphics_toolkit ('gnuplot')

warning('off','all');

rand('state',123456)
exp_name = 'ii2_protein_temp10';

executing gpml startup script...


%load(exp_name)

In [2]:
nn = 40000; 
nv = 2000
n = nn-nv
nt = 5730
load ('./data/protein.mat');
[ndata, ~] = size(M);
X = M(:,2:9);
Y = M(:,1);
R = randperm(ndata);        
ori_xvec_test = X(R(1:nt),:);
ori_yvec_test = Y(R(1:nt));
R(1:nt) = [];
ori_xvec_val = X(R(1:nv),:);          
ori_yvec_val = Y(R(1:nv));
R(1:nv) = [];
ori_xvec = X(R,:);
ori_yvec = Y(R);

nv =  2000
n =  38000
nt =  5730


In [3]:
ttcs = 400;
ttm = round(n / (ttcs/2));
% fast_induce_step = 50;
induce_step = 100;

xnorm = 1; ynorm = 1;

In [4]:
% input normalization
if xnorm == 1
    norm_xmean = mean(ori_xvec);
    norm_xstd = std(ori_xvec);
    xvec = (ori_xvec - repmat(norm_xmean,n,1)) ./ repmat(norm_xstd,n,1);    
    xvec_test = (ori_xvec_test - repmat(norm_xmean,nt,1)) ./ repmat(norm_xstd,nt,1);
    xvec_val = (ori_xvec_val - repmat(norm_xmean,nv,1)) ./ repmat(norm_xstd,nv,1);
else
    xvec = ori_xvec;
    xvec_test = ori_xvec_test;
    xvec_val = ori_xvec_val;
end

In [5]:
% output normalization
if ynorm == 1
    norm_fmean = mean(ori_yvec);
    norm_fstd = std(ori_yvec);
    yvec = (ori_yvec - norm_fmean) / norm_fstd;
    yvec_test = (ori_yvec_test - norm_fmean) / norm_fstd;
    yvec_val = (ori_yvec_val - norm_fmean) / norm_fstd;
else
    yvec = ori_yvec;
    yvec_test = ori_yvec_test;
    yvec_test_val = ori_yvec_val;
end

In [6]:
grls = 0.6:0.01:1.0; % percentage of remaining data to be test

In [7]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(xvec,2);
opts.Xnorm = 'N' ; opts.Ynorm = 'N' ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

meanfunc = [];                    % empty: don't use a mean function
covfunc = opts.covfunc;              % Squared Exponental covariance function
likfunc = opts.likfunc;              % Gaussian likelihood
inffunc = opts.inffunc;
km_iters = 1e4;
opts.induce_step = induce_step ;

In [8]:
kti = 1;
grbcm0_smse_rec = zeros(kti, 1);
vfe0_smse_rec = zeros(kti, 1);
grbcm0_msll_rec = zeros(kti, 1);
vfe0_msll_rec = zeros(kti, 1);

grbcm_smse_rec = zeros(kti, 1);
vfe_smse_rec = zeros(kti, 1);
grbcm_msll_rec = zeros(kti, 1);
vfe_msll_rec = zeros(kti, 1);

In [9]:
% use validation data to find the optimal gr
best_msll = inf;
best_msll_gr = 1;
best_smse = inf;
best_smse_gr = 1;
ki = 1;

# find ratio

In [10]:
dcs = round(ttcs/2);
m = ttm;

n_per = dcs ; % size of Dc

In [11]:
Indics = randperm(n) ;
I_com = Indics(1:n_per) ; % randomly select communication set
[idx, C] = kmeans(xvec, m,'MaxIter',km_iters);    

% hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];    
opts.numOptFC = 30 ;
opts.Ms = m+1;
opts.xvec = xvec;
opts.yvec = yvec;
opts.induce_size = dcs;
opts.grbcm_baseline = 0;
opts.global_index = ones(n,1);
opts.I_com = I_com;
% opts.inffunc = @infGaussLik; opts.meanfunc = meanfunc; opts.likfunc = likfunc;
opts.covfunc = covfunc;
covfuncF = {@apxSparse, {opts.covfunc}, xvec(I_com,:)};
opts.covfuncF = covfuncF;
opts.compute_hyp = 0;

In [12]:
g_opts = opts;
g_opts.compute_hyp = 1;
g_opts.grbcm_baseline = 1;
g_opts.global_index = ones(n,1);
g_models = aggregation_train_GRBCM_VS_apx(xvec,yvec,idx,g_opts); 
opts.hyp = g_models{1}.hyp;
g_opts.hyp = g_models{1}.hyp;
[tmu,ts2, ~] =  aggregation_predict(xvec_val,g_models,'GRBCM', 1, g_opts);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[grbcmMSE,grbcmSMSE,grbcmMSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_val, ori_yvec_val, tmu, ts2);
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'GRBCM', n_per, grbcmMSE,grbcmSMSE,grbcmMSLL);
% grbcm0_smse_rec(ki) = grbcmSMSE; grbcm0_msll_rec(ki) = grbcmMSLL;
g_opts.compute_hyp = 0;

Optimizing hyps in training...
Linesearch     30;  Value 3.574096e+04
GRBCM (Dc size 200): MSE 17.86302354, SMSE 0.45812988, MSLL -0.49297094


In [13]:
vfe_opts = opts;
vfe_opts.induce_type = 'VFE_opt';
xu = xvec(I_com, :);
inffunc = @(varargin) infGaussLik(varargin{:}, struct('s', 0.0));
vfe_hyp = opts.hyp;
vfe_hyp.xu = xu;
[vfe_hyp, tmp_nlzs] = minimize(vfe_hyp,@sp_gp,-vfe_opts.induce_step,inffunc,meanfunc,covfuncF,likfunc,xvec,yvec);
vfe_opts.hyp = opts.hyp;
vfe_opts.xu = vfe_hyp.xu;
vfe_opts.inffunc = @infGaussLik; vfe_opts.meanfunc = meanfunc; vfe_opts.covfuncF = covfuncF; vfe_opts.likfunc = likfunc;
vfe_opts.covfunc = covfunc;
[tmu, ts2] = gp(vfe_hyp, @infGaussLik, meanfunc, covfuncF, likfunc, xvec, yvec, xvec_val);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[vfeMSE,vfeSMSE,vfeMSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_val, ori_yvec_val, tmu, ts2);
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'VFE', n_per, vfeMSE,vfeSMSE,vfeMSLL);
% vfe0_smse_rec(ki) = vfeSMSE; vfe0_msll_rec(ki) = vfeMSLL;
[yu, su] = gp(vfe_hyp, @infGaussLik, meanfunc, covfuncF, likfunc, xvec, yvec, vfe_opts.xu);
vfe_opts.yu = yu; vfe_opts.su = su; 

Function evaluation     99;  Value 2.539783e+05
VFE (Dc size 200): MSE 26.27981368, SMSE 0.67399385, MSLL -0.23702933


In [14]:
models = aggregation_train_GRBCM_VS_apx(xvec,yvec,idx,vfe_opts); % use hyp of vfe
[tmu,ts2] =  aggregation_predict_GRBCM_VS_apx(xvec_val,models,vfe_opts);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[MSE,SMSE,MSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_val, ori_yvec_val, tmu, ts2);
fprintf('%s (Dc size %d): \r\nMSE %6.8f, SMSE %6.4f, MSLL %6.4f\r\n', 'GRBCM++ (VFE)', n_per, MSE,SMSE,MSLL);

In [15]:
sig_temp = 10;
dcs_ecs_r = sigmoid(-MSLL+grbcmMSLL, sig_temp)
dcs = round(ttcs*dcs_ecs_r)
ecs = ttcs - dcs
m = round(n / ecs)

dcs_ecs_r =  0.55502
dcs =  222
ecs =  178
m =  213


# New unbalanced

In [16]:
n_per = dcs ; % size of Dc
Indics = randperm(n) ;
I_com = Indics(1:n_per) ; % randomly select communication set
[idx, C] = kmeans(xvec, m,'MaxIter',km_iters);    

% hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];    
opts.numOptFC = 30 ;
opts.Ms = m+1;
opts.xvec = xvec;
opts.yvec = yvec;
opts.induce_size = dcs;
opts.grbcm_baseline = 0;
opts.global_index = ones(n,1);
opts.I_com = I_com;
% opts.inffunc = @infGaussLik; opts.meanfunc = meanfunc; opts.likfunc = likfunc;
opts.covfunc = covfunc;
covfuncF = {@apxSparse, {opts.covfunc}, xvec(I_com,:)};
opts.covfuncF = covfuncF;
opts.compute_hyp = 0;

In [17]:
g_opts = opts;
g_opts.compute_hyp = 0;
g_opts.grbcm_baseline = 1;
g_opts.global_index = ones(n,1);
g_models = aggregation_train_GRBCM_VS_apx(xvec,yvec,idx,g_opts); 
opts.hyp = g_models{1}.hyp;
g_opts.hyp = g_models{1}.hyp;
[tmu,ts2, ~] =  aggregation_predict(xvec_test,g_models,'GRBCM', 1, g_opts);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[grbcmMSE,grbcmSMSE,grbcmMSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'GRBCM', n_per, grbcmMSE,grbcmSMSE,grbcmMSLL);
% grbcm0_smse_rec(ki) = grbcmSMSE; grbcm0_msll_rec(ki) = grbcmMSLL;
g_opts.compute_hyp = 0;

In [18]:
% % VFE Baseline
vfe_opts = opts;
vfe_opts.induce_type = 'VFE_opt';
xu = xvec(I_com, :);
vfe_hyp = opts.hyp;
vfe_hyp.xu = xu;
vfe_hyp = minimize(vfe_hyp,@sp_gp,-vfe_opts.induce_step,inffunc,meanfunc,covfuncF,likfunc,xvec,yvec);
vfe_opts.hyp = opts.hyp;
vfe_opts.xu = vfe_hyp.xu;
vfe_opts.inffunc = @infGaussLik; vfe_opts.meanfunc = meanfunc; vfe_opts.covfuncF = covfuncF; vfe_opts.likfunc = likfunc;
vfe_opts.covfunc = covfunc;
[tmu, ts2] = gp(vfe_hyp, @infGaussLik, meanfunc, covfuncF, likfunc, xvec, yvec, xvec_test);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[vfeMSE,vfeSMSE,vfeMSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'VFE baseline', n_per, vfeMSE,vfeSMSE,vfeMSLL);
% vfe0_smse_rec(ki) = vfeSMSE; vfe0_msll_rec(ki) = vfeMSLL;
[yu, su] = gp(vfe_hyp, @infGaussLik, meanfunc, covfuncF, likfunc, xvec, yvec, vfe_opts.xu);
vfe_opts.yu = yu; vfe_opts.su = su; 

Function evaluation     99;  Value 2.506688e+05
VFE baseline (Dc size 222): MSE 24.46594705, SMSE 0.65215221, MSLL -0.25519250


In [19]:
sp_opts = opts;
sp_opts.induce_type = 'SPGP_opt';
hyp_init(1:d,1) = -2*opts.hyp.cov(1:d);
hyp_init(d+1,1) = 2*opts.hyp.cov(d+1);
hyp_init(d+2,1) = 2*opts.hyp.lik;

sp_opts.induce_size = dcs;
xu = xvec(I_com, :);
w_init = [reshape(xu,sp_opts.induce_size*d,1);hyp_init];
[w,f] = minimize(w_init,'spgp_lik_nohyp',-sp_opts.induce_step,yvec,xvec,sp_opts.induce_size);
xb = reshape(w(1:sp_opts.induce_size*d,1),sp_opts.induce_size,d);
sp_opts.xu = xb;
sp_opts.sp_hyp = w(sp_opts.induce_size*d+1:end,1);
sp_opts.hyp = opts.hyp;

[tmu,ts2] = spgp_pred(sp_opts.yvec,sp_opts.xvec,sp_opts.xu,xvec_test,sp_opts.sp_hyp);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[spgpMSE,spgpSMSE,spgpMSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'SPSG baseline', n_per, spgpMSE,spgpSMSE,spgpMSLL);
% spgp0_smse_rec(ki) = spgpSMSE; spgp0_msll_rec(ki) = spgpMSLL;
[yu,su] = spgp_pred(sp_opts.yvec,sp_opts.xvec,sp_opts.xu,sp_opts.xu,sp_opts.sp_hyp);
sp_opts.yu = yu; sp_opts.su = su;

Function evaluation     99;  Value 4.461679e+04
SPSG baseline (Dc size 222): MSE 25.26604283, SMSE 0.67347917, MSLL -0.23982141


In [20]:
disp('validation results')

validation results


In [21]:
% compute informatic importance
ds = zeros(n, m);
for i=1:n
    for j=1:m
        %ds(i,j) = muti1(i)-muti2(j,i)+muti1(j)-muti2(i,j);
        ds(i,j) = norm(xvec(i, :) - C(j,:));
    end
end
% 1st closest cluster centers
[~, fcc] = min(ds, [], 2);
% set to inf
for i=1:n
    ds(i, fcc(i)) = 1e10;
end
% 2st closest cluster centers
[~, scc] = min(ds, [], 2);
rk = zeros(n, 1);
for iik=1:n
    if mod(iik, 1e2)==0
        fprintf('processing distance: %d/%d\n', iik, n);
    end
    i = fcc(iik);
    [tmp_mu, tmp_sig2] = gp(opts.hyp,inffunc,meanfunc, ...
        covfunc,likfunc,g_models{i+1}.X,g_models{i+1}.Y,xvec(iik,:));
    h1 = 0.5*log(tmp_sig2);
    j = scc(iik);
%     [tmp_mu, tmp_sig2] = gp(opts.hyp,inffunc,meanfunc, ...
%         covfunc,likfunc,[g_models{i+1}.X;g_models{j+1}.X],[g_models{i+1}.Y;g_models{j+1}.Y],xvec(iik,:));
    [tmp_mu, tmp_sig2] = gp(opts.hyp,inffunc,meanfunc, ...
        covfunc,likfunc,[g_models{j+1}.X],[g_models{j+1}.Y],xvec(iik,:));
    h2 = 0.5*log(tmp_sig2);
    rk(iik) = h2 - h1; % importance is the difference between mutual information
end

processing distance: 100/38000
processing distance: 200/38000
processing distance: 300/38000
processing distance: 400/38000
processing distance: 500/38000
processing distance: 600/38000
processing distance: 700/38000
processing distance: 800/38000
processing distance: 900/38000
processing distance: 1000/38000
processing distance: 1100/38000
processing distance: 1200/38000
processing distance: 1300/38000
processing distance: 1400/38000
processing distance: 1500/38000
processing distance: 1600/38000
processing distance: 1700/38000
processing distance: 1800/38000
processing distance: 1900/38000
processing distance: 2000/38000
processing distance: 2100/38000
processing distance: 2200/38000
processing distance: 2300/38000
processing distance: 2400/38000
processing distance: 2500/38000
processing distance: 2600/38000
processing distance: 2700/38000
processing distance: 2800/38000
processing distance: 2900/38000
processing distance: 3000/38000
processing distance: 3100/38000
processing distan

processing distance: 25200/38000
processing distance: 25300/38000
processing distance: 25400/38000
processing distance: 25500/38000
processing distance: 25600/38000
processing distance: 25700/38000
processing distance: 25800/38000
processing distance: 25900/38000
processing distance: 26000/38000
processing distance: 26100/38000
processing distance: 26200/38000
processing distance: 26300/38000
processing distance: 26400/38000
processing distance: 26500/38000
processing distance: 26600/38000
processing distance: 26700/38000
processing distance: 26800/38000
processing distance: 26900/38000
processing distance: 27000/38000
processing distance: 27100/38000
processing distance: 27200/38000
processing distance: 27300/38000
processing distance: 27400/38000
processing distance: 27500/38000
processing distance: 27600/38000
processing distance: 27700/38000
processing distance: 27800/38000
processing distance: 27900/38000
processing distance: 28000/38000
processing distance: 28100/38000
processing

In [22]:
for kj=1:length(grls) % test/validate different remaining percentage
    kj
    gr=grls(kj)

    % remove data, and use sparse approximation
    crk = rk;
    [~, crk_idx] = sort(crk, 'descend');
    rn = round(n*gr);
    global_index = crk_idx(1:rn);
    global_index = zeros(n,1);
    global_index(crk_idx(1:rn)) = 1; % select remaining data according to the importance
    
    vfe_opts.global_index = global_index;
    models = aggregation_train_GRBCM_VS_apx(xvec,yvec,idx,vfe_opts); % use hyp of vfe
    [tmu,ts2] = aggregation_predict_GRBCM_VS_apx(xvec_val,models,vfe_opts);
    if ynorm==1
        tmu = tmu * norm_fstd + norm_fmean;
        ts2 = ts2 * norm_fstd^2;
    end
    [MSE,SMSE,MSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_val, ori_yvec_val, tmu, ts2);
    fprintf('%s (Dc size %d): \r\nMSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'GRBCM++', n_per, MSE,SMSE,MSLL);
    vfe_gr_smse(ki,kj) = SMSE; vfe_gr_msll(ki,kj) = MSLL;
    if MSLL < best_msll
        best_msll = MSLL; best_msll_gr = gr;
    end
    if SMSE < best_smse
        best_smse = SMSE; best_smse_gr = gr;
    end
    
    sp_opts.global_index = global_index;
    models = aggregation_train_GRBCM_VS_apx(xvec,yvec,idx,sp_opts); % use hyp of vfe
    [tmu,ts2] = aggregation_predict_GRBCM_VS_apx(xvec_val,models,sp_opts);
    if ynorm==1
        tmu = tmu * norm_fstd + norm_fmean;
        ts2 = ts2 * norm_fstd^2;
    end
    [MSE,SMSE,MSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_val, ori_yvec_val, tmu, ts2);
    fprintf('%s (Dc size %d): \r\nMSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'GRBCM++', n_per, MSE,SMSE,MSLL);
    spgp_gr_smse(ki,kj) = SMSE; spgp_gr_msll(ki,kj) = MSLL;
end   

kj =  1
gr =  0.60000
GRBCM++ (Dc size 222): 
MSE 19.08885907, SMSE 0.48956867, MSLL -0.50476238
GRBCM++ (Dc size 222): 
MSE 19.37353499, SMSE 0.49686971, MSLL -0.49450037
kj =  2
gr =  0.61000
GRBCM++ (Dc size 222): 
MSE 18.98642623, SMSE 0.48694160, MSLL -0.50722361
GRBCM++ (Dc size 222): 
MSE 19.25769756, SMSE 0.49389885, MSLL -0.49697663
kj =  3
gr =  0.62000
GRBCM++ (Dc size 222): 
MSE 18.90181691, SMSE 0.48477164, MSLL -0.51104696
GRBCM++ (Dc size 222): 
MSE 19.15551030, SMSE 0.49127807, MSLL -0.50080540
kj =  4
gr =  0.63000
GRBCM++ (Dc size 222): 
MSE 18.83667408, SMSE 0.48310093, MSLL -0.51366646
GRBCM++ (Dc size 222): 
MSE 19.08693990, SMSE 0.48951945, MSLL -0.50363985
kj =  5
gr =  0.64000
GRBCM++ (Dc size 222): 
MSE 18.81143236, SMSE 0.48245356, MSLL -0.51530258
GRBCM++ (Dc size 222): 
MSE 19.06779422, SMSE 0.48902843, MSLL -0.50533468
kj =  6
gr =  0.65000
GRBCM++ (Dc size 222): 
MSE 18.70471217, SMSE 0.47971652, MSLL -0.51951162
GRBCM++ (Dc size 222): 
MSE 18.96006465, SM

In [23]:
fprintf('best msll gr %6.8f, best MSLL %6.8f\r\n', best_msll_gr, best_msll);
fprintf('best smse gr %6.8f, best SMSE %6.8f\r\n', best_smse_gr, best_smse);

best msll gr 0.94000000, best MSLL -0.53829054
best smse gr 1.00000000, best SMSE 0.44559428


In [24]:
for ki=1:kti
    for gr = [best_msll_gr, best_smse_gr, 1.0]
        gr
        crk = rk;
        crk(I_com) = -1e10;
        [~, crk_idx] = sort(crk, 'descend');
        rn = round(n*gr);
        global_index = zeros(n,1);
        global_index(crk_idx(1:rn)) = 1; % select remaining data according to the importance
           
        vfe_opts.global_index = global_index;
        models = aggregation_train_GRBCM_VS_apx(xvec,yvec,idx,vfe_opts); % use hyp of vfe
        [tmu,ts2] =  aggregation_predict_GRBCM_VS_apx(xvec_test,models,vfe_opts);
        if ynorm==1
            tmu = tmu * norm_fstd + norm_fmean;
            ts2 = ts2 * norm_fstd^2;
        end
        [MSE,SMSE,MSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);
        fprintf('%s (Dc size %d): \r\nMSE %6.8f, SMSE %6.4f, MSLL %6.4f\r\n', 'GRBCM++ (VFE)', n_per, MSE,SMSE,MSLL);
%         grbcm2_gr_smse(ki,kj) = SMSE; grbcm2_gr_msll(ki,kj) = MSLL;
        
        sp_opts.global_index = global_index;
        models = aggregation_train_GRBCM_VS_apx(xvec,yvec,idx,sp_opts); % use hyp of vfe
        [tmu,ts2] =  aggregation_predict_GRBCM_VS_apx(xvec_test,models,sp_opts);
        if ynorm==1
            tmu = tmu * norm_fstd + norm_fmean;
            ts2 = ts2 * norm_fstd^2;
        end
        [MSE,SMSE,MSLL] = evaluate2(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);
        fprintf('%s (Dc size %d): \r\nMSE %6.8f, SMSE %6.4f, MSLL %6.4f\r\n', 'GRBCM++ (SPGP)', n_per, MSE,SMSE,MSLL);
%         grbcm2_spgp_gr_smse(ki,kj) = SMSE; grbcm2_spgp_gr_msll(ki,kj) = MSLL;
    end   
end

gr =  0.94000
GRBCM++ (VFE) (Dc size 222): 
MSE 15.97119087, SMSE 0.4257, MSLL -0.6032
GRBCM++ (SPGP) (Dc size 222): 
MSE 16.00519948, SMSE 0.4266, MSLL -0.5911
gr =  1
GRBCM++ (VFE) (Dc size 222): 
MSE 15.79648705, SMSE 0.4211, MSLL -0.6097
GRBCM++ (SPGP) (Dc size 222): 
MSE 15.83617191, SMSE 0.4221, MSLL -0.5969
gr =  1
GRBCM++ (VFE) (Dc size 222): 
MSE 15.79648705, SMSE 0.4211, MSLL -0.6097
GRBCM++ (SPGP) (Dc size 222): 
MSE 15.83617191, SMSE 0.4221, MSLL -0.5969


In [25]:
save(exp_name)